In [37]:
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import random
from matplotlib import pyplot as plt
from keras.wrappers.scikit_learn import KerasClassifier
from functions import *
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from copy import deepcopy as dp
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import categorical_accuracy
import os
from keras import backend as K 
import optuna
from keras.optimizers import Adam
def print_cmx(y_true, y_pred):
    cmx_data = confusion_matrix(y_true, y_pred)
    
    df_cmx = pd.DataFrame(cmx_data, index=['hate','offend','neither'], columns=['hate','offend','neither'])
    df_cmx = df_cmx.div(df_cmx.sum(axis=1), axis=0)*100
    
    plt.figure(figsize = (10,7))
    sns.heatmap(df_cmx, annot=True)
    plt.show()
    return df_cmx

def tokenize(text):
    return text.split(' ')

In [38]:
def gen_model(hidden, drop, vocab_size, embed):
    model = Sequential()
    model.add(Embedding(vocab_size, 20, input_length = embed, mask_zero = True))
    model.add(LSTM(hidden))
    model.add(Dropout(drop))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    
    return model

In [14]:
import warnings
warnings.filterwarnings('ignore')
optuna.logging.enable_default_handler()

In [15]:
data, w_i, i_d = load_pkl(r'C:\Users\HEndo\Documents\GitHub\zemi\cleaned_corpus\base_model_input.pkl')
test_a, train_a = load_pkl(r'C:\Users\HEndo\Documents\GitHub\zemi\cleaned_corpus\answer_data.pkl')
test, train = data

sets = list(zip(train, train_a))
random.shuffle(sets)
hate = [text for text in sets if np.argmax(text[1]) == 0]
off = [text for text in sets if np.argmax(text[1]) == 1]
nei = [text for text in sets if np.argmax(text[1]) == 2]
train_set = []
valid_set = []
for i in [hate, off, nei]:
    random.shuffle(i)
    train_set.extend(i[round(len(i)*0.2):])
    valid_set.extend(i[:round(len(i)*0.2)])

In [16]:
random.shuffle(train_set)
train_X, train_y = zip(*train_set)
valid_X, valid_y = zip(*valid_set)

In [17]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, min_delta=0.005)


In [18]:
vocab_size = len(w_i)
embed_size = len(train_X[0])

In [19]:
from keras.backend import tensorflow_backend as backend


In [20]:
def objective(trial):
    random.shuffle(train_set)
    train_X, train_y = zip(*train_set)
    params = {
    'lr': trial.suggest_loguniform('lr', 1e-3, 1e-1),
    'hidden': trial.suggest_categorical('hidden', list(range(1, 128, 10))),
    'drop': trial.suggest_uniform('drop', 0.1, 1),
    'batch': trial.suggest_categorical('batch', [18, 52, 104]),
    }
    adam = Adam(lr=params['lr'])
    model = gen_model(params['hidden'], params['drop'], vocab_size= vocab_size, embed=embed_size) 
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=[categorical_accuracy])
    history = model.fit(np.array(train_X), np.array(train_y), batch_size=params['batch'], 
                        validation_data = (np.array(valid_X), np.array(valid_y)), 
                        epochs=10, shuffle=True, verbose = 0,
                        callbacks = [es]#, KerasPruningCallback(trial, 'val_acc')]
                       )
    pred = np.argmax(model.predict(np.array(valid_X)), axis = 1)
    true = np.argmax(valid_y, axis = 1)
    score = f1_score(y_true=true, y_pred=pred, average = 'macro')
    backend.clear_session()
    history
    return 1.0-score

In [21]:
study = optuna.create_study()
study.optimize(objective, n_trials=255, n_jobs=1, verb_pace = 10)

Epoch 00005: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00008: early stopping


C:\Users\HEndo\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 00006: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00007: early stopping
Epoch 00004: early stopping


[I 2019-02-26 22:13:03,162] Finished trial 10 / 255. Current best value is 0.22131426379496144 with parameters: {'lr': 0.01433989347151844, 'hidden': 111, 'drop': 0.4956600417439637, 'batch': 18}.


Epoch 00005: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping


...

Epoch 00007: early stopping


C:\Users\HEndo\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 00006: early stopping


C:\Users\HEndo\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping


In [3]:
best = {'lr': 0.0052365668686423705, 'hidden': 91, 'drop': 0.4732293175092074, 'batch': 18}
save_pkl('tuned_parameters/lstm.pkl', best)

In [4]:
cw = {0:3, 1:1, 2:1.25}

In [23]:
mcp_save = ModelCheckpoint('best_lstm2.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [39]:
def objective_class_weight(trial):
    random.shuffle(train_set)
    train_X, train_y = zip(*train_set)
    params = {
    'lr': trial.suggest_loguniform('lr', 1e-3, 1e-1),
    'hidden': trial.suggest_categorical('hidden', list(range(1, 128, 10))),
    'drop': trial.suggest_uniform('drop', 0.1, 1),
    'batch': 52#trial.suggest_categorical('batch', [18, 52, 104]),
    }
    adam = Adam(lr=params['lr'])
    model = gen_model(params['hidden'], params['drop'], vocab_size= vocab_size, embed=embed_size) 
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=[categorical_accuracy])
    history = model.fit(np.array(train_X), np.array(train_y), batch_size=params['batch'], 
                        validation_data = (np.array(valid_X), np.array(valid_y)), 
                        epochs=6, shuffle=True, verbose = 0,
                        callbacks = [es, mcp_save], #, KerasPruningCallback(trial, 'val_acc')]
                        class_weight = cw
                       )
    model = load_model('best_lstm2.hdf5')
    pred = np.argmax(model.predict(np.array(valid_X)), axis = 1)
    true = np.argmax(valid_y, axis = 1)
    score = f1_score(y_true=true, y_pred=pred, average = 'macro')
    backend.clear_session()
    history
    return 1.0-score

In [40]:
study_cw = optuna.create_study()
study_cw.optimize(objective_class_weight, n_trials=200, n_jobs=1, verb_pace = 10)

Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping


[I 2019-02-28 17:01:04,552] Finished trial 10 / 200. Current best value is 0.20511729408045853 with parameters: {'lr': 0.003938782895310286, 'hidden': 81, 'drop': 0.5613009364853403}.


Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00003: early stopping


[I 2019-02-28 17:06:55,373] Finished trial 20 / 200. Current best value is 0.20511729408045853 with parameters: {'lr': 0.003938782895310286, 'hidden': 81, 'drop': 0.5613009364853403}.


Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping


...

Epoch 00003: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00006: early stopping
Epoch 00003: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping


[I 2019-02-28 18:43:40,742] Finished trial 190 / 200. Current best value is 0.203146663265799 with parameters: {'lr': 0.0016506045934272253, 'hidden': 41, 'drop': 0.3159095644935586}.


Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00003: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping
Epoch 00004: early stopping
Epoch 00005: early stopping


[I 2019-02-28 18:49:01,285] Finished trial 200 / 200. Current best value is 0.203146663265799 with parameters: {'lr': 0.0016506045934272253, 'hidden': 41, 'drop': 0.3159095644935586}.


Epoch 00004: early stopping


In [41]:
best_cw = dp(study_cw.best_params)
best_cw['batch'] = 52
save_pkl('tuned_parameters/lstm_class_weight.pkl', best_cw)